In [1]:
import sys
import os
from pathlib import Path

sys.path.append(str(Path(sys.argv[0]).absolute().parent.parent.parent.parent))
base_repo = os.path.realpath(os.path.join(os.getcwd(), "../../"))
print(f"{base_repo=}")

sys.path.append(base_repo)

# add the entire folder to path
print(f"{sys.path=}")
print(f"{os.getcwd()=}")

base_repo='E:\\Development\\bot-detector\\remote\\bot-detector-ml-training'
sys.path=['e:\\Development\\bot-detector\\remote\\bot-detector-ml-training\\src\\discovery', 'D:\\Users\\creyfye\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'D:\\Users\\creyfye\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'D:\\Users\\creyfye\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'D:\\Users\\creyfye\\AppData\\Local\\Programs\\Python\\Python311', 'e:\\Development\\bot-detector\\remote\\bot-detector-ml-training\\.venv', '', 'e:\\Development\\bot-detector\\remote\\bot-detector-ml-training\\.venv\\Lib\\site-packages', 'e:\\Development\\bot-detector\\remote\\bot-detector-ml-training\\.venv\\Lib\\site-packages\\win32', 'e:\\Development\\bot-detector\\remote\\bot-detector-ml-training\\.venv\\Lib\\site-packages\\win32\\lib', 'e:\\Development\\bot-detector\\remote\\bot-detector-ml-training\\.venv\\Lib\\site-packages\\Pythonwin', 'e:\\Development\\bot-detector\\remote\\bot-detector-

In [2]:
from src.utils import utils
from src.utils import data
from src.utils import classifier

import collections
import os.path

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import tree

import graphviz

In [3]:
PATH_TO_FILE = f"../data/raw/2022-10-26_hiscore_data.parquet.gzip"

df = pd.read_parquet(PATH_TO_FILE)
df = df.reset_index()
df.head()

,Player_id,name,created_at,updated_at,possible_ban,confirmed_ban,label_id,label,account_status,id,...,tombs_of_amascut,tombs_of_amascut_expert,tzkal_zuk,tztok_jad,venenatis,vetion,vorkath,wintertodt,zalcano,zulrah
0,1,3BA604236FB0319D5937E31388B0C64C,2021-03-14 20:22:45,2022-12-19 05:02:57,0,0,1,Real_Player,not banned,59568395,...,0.0,0.0,0,0,0,0,0,0,0,0
1,8,5A02B5A7F38AD2623A9C5E68DF01EC2F,2021-03-14 20:42:37,2022-12-19 00:36:08,0,0,1,Real_Player,not banned,59622273,...,0.0,0.0,0,10,0,0,114,73,0,1256
2,29,59DCFCAFC1F3DF3326F36E7A39B741FC,2021-03-14 22:17:16,2022-12-19 14:49:19,0,0,1,Real_Player,not banned,292513577,...,0.0,0.0,0,0,0,0,0,0,0,0
3,39,1C74EFD6CE51790D7BF65A94F47675B5,2021-03-14 22:17:23,2022-12-19 17:46:42,0,0,1,Real_Player,not banned,59503415,...,235.0,0.0,0,0,0,0,279,1526,145,0
4,59,E666957B20A95519E6306D75FEC4DE19,2021-03-14 22:17:40,2022-12-19 01:23:32,1,0,1,Real_Player,not banned,59615490,...,0.0,0.0,0,20,0,0,1000,500,0,400


In [4]:
OUTPUT_DIR = os.path.realpath("../data/output")
os.chdir(OUTPUT_DIR)

In [5]:
# This logic is roughly equivalent to app.py's train function
hiscoredata = data.hiscoreData(df, False)
features = hiscoredata.features()

In [6]:
binary_classifier = classifier.classifier("binaryClassifier")
dt_binary_classifier = classifier.DTclassifier("DTbinaryClassifier")

In [7]:
x = features.copy()
x = x.drop(
    columns=[
        "label",
        "label_id",
        "name",
        "created_at",
        "updated_at",
        "account_status",
        "possible_ban",
        "confirmed_ban",
    ]
)
print("x columns:\n", x.columns)
x.head()

x columns:
 Index(['total', 'attack', 'defence', 'strength', 'hitpoints', 'ranged',
       'prayer', 'magic', 'cooking', 'woodcutting',
       ...
       'tombs_of_amascut/total', 'tombs_of_amascut_expert/total',
       'tzkal_zuk/total', 'tztok_jad/total', 'venenatis/total', 'vetion/total',
       'vorkath/total', 'wintertodt/total', 'zalcano/total', 'zulrah/total'],
      dtype='object', length=160)


,total,attack,defence,strength,hitpoints,ranged,prayer,magic,cooking,woodcutting,...,tombs_of_amascut/total,tombs_of_amascut_expert/total,tzkal_zuk/total,tztok_jad/total,venenatis/total,vetion/total,vorkath/total,wintertodt/total,zalcano/total,zulrah/total
Player_id,,,,,,,,,,,,,,,,,,,,,
1,55978987,5491118,3358410,5789019,5237822,1877958,748453,3332131,13050181,947390,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
8,249164755,14151880,13293654,15162818,25533323,19180307,13078638,14765780,13057071,13102209,...,0.000000,0.0,0.0,0.004093,0.0,0.0,0.046661,0.029877,0.000000,0.514160
29,25751694,674007,0,3150582,2971492,2595141,131224,8527168,288072,442417,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
39,330938656,14153462,13409025,14231093,23612757,20071372,13366234,12044152,15746067,9140223,...,0.038452,0.0,0.0,0.000000,0.0,0.0,0.045654,0.249756,0.023727,0.000000
59,380812843,25789847,19279215,57396170,51308981,45806374,13037612,20373613,13144778,13051121,...,0.000000,0.0,0.0,0.001207,0.0,0.0,0.060364,0.030182,0.000000,0.024155


In [8]:
x.describe()

,total,attack,defence,strength,hitpoints,ranged,prayer,magic,cooking,woodcutting,...,tombs_of_amascut/total,tombs_of_amascut_expert/total,tzkal_zuk/total,tztok_jad/total,venenatis/total,vetion/total,vorkath/total,wintertodt/total,zalcano/total,zulrah/total
count,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,...,286234.0,286234.000000,286234.0,286234.0,286234.0,286234.0,286234.0,286234.00000,286234.0,286234.0
mean,7.592381e+07,4.613691e+06,3.989119e+06,6.834497e+06,8.833890e+06,8.013346e+06,1.866151e+06,4.951919e+06,3.780524e+06,2.505234e+06,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
std,1.523338e+08,9.737696e+06,8.368783e+06,1.532521e+07,1.779451e+07,1.819261e+07,5.276381e+06,9.678008e+06,1.258254e+07,7.865430e+06,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
25%,2.589612e+06,1.378500e+04,4.480000e+03,1.375000e+04,1.623500e+04,1.200000e+01,1.424000e+03,1.859000e+03,3.700000e+02,1.800000e+03,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
50%,1.618213e+07,7.376685e+05,5.968565e+05,8.152260e+05,1.211640e+06,4.932785e+05,1.018635e+05,8.367410e+05,1.808900e+05,2.140325e+05,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
75%,8.187651e+07,5.113342e+06,4.208670e+06,8.626339e+06,1.003393e+07,7.466200e+06,1.262802e+06,6.401728e+06,2.704568e+06,1.475885e+06,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.01461,0.0,0.0
max,4.600000e+09,2.000000e+08,2.000000e+08,2.000000e+08,2.000000e+08,2.000000e+08,2.000000e+08,2.000000e+08,2.000000e+08,2.000000e+08,...,1.0,0.988281,1.0,1.0,1.0,1.0,1.0,1.00000,1.0,1.0


In [9]:
y = features["label"].apply(lambda x: 0 if x == "Real_Player" else 1)
y.head()

Player_id
1     0
8     0
29    0
39    0
59    0
Name: label, dtype: int64

In [10]:
def fit_score(classifier, x, y, class_names=None, rpt=True):
    # This logic is roughly equivalent to app.py's train function
    train_x, test_x, train_y, test_y = train_test_split(
        x, y, test_size=0.2, random_state=42, stratify=y
    )
    classifier.fit(train_x, train_y)

    accuracy, roc_auc = classifier.score(test_y, test_x, class_names)
    if rpt:
        print(f"accuracy: {accuracy}, roc_auc: {roc_auc}")
    # OUTPUT: (0.9992126580557206, 0.9992126580557205)

    if rpt:
        print(classification_report(y_true=test_y, y_pred=classifier.predict(test_x)))
    return accuracy


# To inspect binary classifier random forest and decision tree performance:
# fit_score(binary_classifier, x, y)
# fit_score(dt_binary_classifier, x, y)

In [11]:
# Output of block is x_multi, y_multi. Fiddle around with data here.
multi_classifier = classifier.classifier("multiClassifier")
dt_multi_classifier = classifier.DTclassifier("DTmultiClassifier")

In [12]:
y_labels = features["label"].value_counts()
y_labels = [label for label, value in y_labels.items() if value > 500]
y_labels

['Real_Player',
 'Unknown_bot',
 'PVM_Melee_bot',
 'Magic_bot',
 'Smithing_bot',
 'Fishing_bot',
 'Crafting_bot',
 'Zulrah_bot',
 'Mining_bot',
 'Fletching_bot',
 'Hunter_bot',
 'Agility_bot',
 'Woodcutting_bot',
 'Runecrafting_bot',
 'LMS_bot',
 'Zalcano_bot',
 'Herblore_bot',
 'Thieving_bot',
 'Barrows_bot',
 'Soul_Wars_bot',
 'Cooking_bot',
 'Blast_mine_bot',
 'Wintertodt_bot',
 'Vorkath_bot']

In [13]:
x_multi = features.copy()
x_multi = x_multi[x_multi["label"].isin(y_labels)]
x_multi.head()

,name,created_at,updated_at,possible_ban,confirmed_ban,label_id,label,account_status,total,attack,...,tombs_of_amascut/total,tombs_of_amascut_expert/total,tzkal_zuk/total,tztok_jad/total,venenatis/total,vetion/total,vorkath/total,wintertodt/total,zalcano/total,zulrah/total
Player_id,,,,,,,,,,,,,,,,,,,,,
1,3BA604236FB0319D5937E31388B0C64C,2021-03-14 20:22:45,2022-12-19 05:02:57,0,0,1,Real_Player,not banned,55978987,5491118,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
8,5A02B5A7F38AD2623A9C5E68DF01EC2F,2021-03-14 20:42:37,2022-12-19 00:36:08,0,0,1,Real_Player,not banned,249164755,14151880,...,0.000000,0.0,0.0,0.004093,0.0,0.0,0.046661,0.029877,0.000000,0.514160
29,59DCFCAFC1F3DF3326F36E7A39B741FC,2021-03-14 22:17:16,2022-12-19 14:49:19,0,0,1,Real_Player,not banned,25751694,674007,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
39,1C74EFD6CE51790D7BF65A94F47675B5,2021-03-14 22:17:23,2022-12-19 17:46:42,0,0,1,Real_Player,not banned,330938656,14153462,...,0.038452,0.0,0.0,0.000000,0.0,0.0,0.045654,0.249756,0.023727,0.000000
59,E666957B20A95519E6306D75FEC4DE19,2021-03-14 22:17:40,2022-12-19 01:23:32,1,0,1,Real_Player,not banned,380812843,25789847,...,0.000000,0.0,0.0,0.001207,0.0,0.0,0.060364,0.030182,0.000000,0.024155


In [14]:
label_to_id = dict(
    (label, label_id) for label_id, label in enumerate(x_multi["label"].unique())
)
label_to_id

{'Real_Player': 0,
 'Wintertodt_bot': 1,
 'Mining_bot': 2,
 'Hunter_bot': 3,
 'Herblore_bot': 4,
 'Fletching_bot': 5,
 'Fishing_bot': 6,
 'Crafting_bot': 7,
 'Cooking_bot': 8,
 'Woodcutting_bot': 9,
 'Smithing_bot': 10,
 'Magic_bot': 11,
 'Agility_bot': 12,
 'Zalcano_bot': 13,
 'Runecrafting_bot': 14,
 'PVM_Melee_bot': 15,
 'Thieving_bot': 16,
 'LMS_bot': 17,
 'Soul_Wars_bot': 18,
 'Vorkath_bot': 19,
 'Barrows_bot': 20,
 'Unknown_bot': 21,
 'Blast_mine_bot': 22,
 'Zulrah_bot': 23}

In [15]:
y_multi = x_multi["label"]  # .apply(lambda x: label_to_id[x])
y_multi

Player_id
1           Real_Player
8           Real_Player
29          Real_Player
39          Real_Player
59          Real_Player
               ...     
49766405     Zulrah_bot
49775805     Zulrah_bot
49776809     Zulrah_bot
49786285     Zulrah_bot
49835352     Zulrah_bot
Name: label, Length: 285167, dtype: object

In [16]:
enum_label = list(enumerate(y_multi.unique()))
class_names = [t[1] for t in enum_label]
label_to_enum = dict([(k, v) for v, k in enum_label])
label_to_enum

{'Real_Player': 0,
 'Wintertodt_bot': 1,
 'Mining_bot': 2,
 'Hunter_bot': 3,
 'Herblore_bot': 4,
 'Fletching_bot': 5,
 'Fishing_bot': 6,
 'Crafting_bot': 7,
 'Cooking_bot': 8,
 'Woodcutting_bot': 9,
 'Smithing_bot': 10,
 'Magic_bot': 11,
 'Agility_bot': 12,
 'Zalcano_bot': 13,
 'Runecrafting_bot': 14,
 'PVM_Melee_bot': 15,
 'Thieving_bot': 16,
 'LMS_bot': 17,
 'Soul_Wars_bot': 18,
 'Vorkath_bot': 19,
 'Barrows_bot': 20,
 'Unknown_bot': 21,
 'Blast_mine_bot': 22,
 'Zulrah_bot': 23}

In [17]:
x_multi = x_multi.drop(
    columns=[
        "label",
        "label_id",
        "name",
        "created_at",
        "updated_at",
        "account_status",
        "possible_ban",
        "confirmed_ban",
    ]
)
x_multi = x_multi[x_multi.columns.drop(list(x_multi.filter(regex="/total")))]
x_multi

,total,attack,defence,strength,hitpoints,ranged,prayer,magic,cooking,woodcutting,...,tombs_of_amascut_expert,tzkal_zuk,tztok_jad,venenatis,vetion,vorkath,wintertodt,zalcano,zulrah,boss_total
Player_id,,,,,,,,,,,,,,,,,,,,,
1,55978987,5491118,3358410,5789019,5237822,1877958,748453,3332131,13050181,947390,...,0.0,0,0,0,0,0,0,0,0,0
8,249164755,14151880,13293654,15162818,25533323,19180307,13078638,14765780,13057071,13102209,...,0.0,0,10,0,0,114,73,0,1256,2443
29,25751694,674007,0,3150582,2971492,2595141,131224,8527168,288072,442417,...,0.0,0,0,0,0,0,0,0,0,73
39,330938656,14153462,13409025,14231093,23612757,20071372,13366234,12044152,15746067,9140223,...,0.0,0,0,0,0,279,1526,145,0,6111
59,380812843,25789847,19279215,57396170,51308981,45806374,13037612,20373613,13144778,13051121,...,0.0,0,20,0,0,1000,500,0,400,16565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49766405,38899515,1222728,1497164,1563387,7010515,14909991,836761,5077944,841847,620150,...,0.0,0,0,0,0,188,0,0,953,1141
49775805,11432596,450229,867427,650195,1103873,1197300,273873,3995877,410050,172397,...,0.0,0,0,0,0,0,0,0,88,88
49776809,60733963,1813071,1988725,1639713,7752484,17441972,907412,19911039,383915,6859815,...,0.0,0,0,0,0,0,0,0,65,82


In [18]:
print("x_multi columns:\n", x_multi.columns)

x_multi columns:
 Index(['total', 'attack', 'defence', 'strength', 'hitpoints', 'ranged',
       'prayer', 'magic', 'cooking', 'woodcutting', 'fletching', 'fishing',
       'firemaking', 'crafting', 'smithing', 'mining', 'herblore', 'agility',
       'thieving', 'slayer', 'farming', 'runecraft', 'hunter', 'construction',
       'league', 'bounty_hunter_hunter', 'bounty_hunter_rogue', 'cs_all',
       'cs_beginner', 'cs_easy', 'cs_medium', 'cs_hard', 'cs_elite',
       'cs_master', 'lms_rank', 'soul_wars_zeal', 'abyssal_sire',
       'alchemical_hydra', 'barrows_chests', 'bryophyta', 'callisto',
       'cerberus', 'chambers_of_xeric', 'chambers_of_xeric_challenge_mode',
       'chaos_elemental', 'chaos_fanatic', 'commander_zilyana',
       'corporeal_beast', 'crazy_archaeologist', 'dagannoth_prime',
       'dagannoth_rex', 'dagannoth_supreme', 'deranged_archaeologist',
       'general_graardor', 'giant_mole', 'grotesque_guardians', 'hespori',
       'kalphite_queen', 'king_black_dragon'

In [19]:
# Train a decision tree and output info about the set of samples that end up in each leaf node.
# Loosely based on: https://stackoverflow.com/a/66299085/5024503
def retrieve_branches(number_nodes, children_left_list, children_right_list):
    """Retrieve decision tree branches"""

    # Calculate if a node is a leaf
    is_leaves_list = [
        (False if cl != cr else True)
        for cl, cr in zip(children_left_list, children_right_list)
    ]

    # Store the branches paths
    paths = []

    for i in range(number_nodes):
        if is_leaves_list[i]:
            # Search leaf node in previous paths
            end_node = [path[-1] for path in paths]

            # If it is a leave node yield the path
            if i in end_node:
                output = paths.pop(np.argwhere(i == np.array(end_node))[0][0])
                yield output

        else:
            # Origin and end nodes
            origin, end_l, end_r = i, children_left_list[i], children_right_list[i]

            # Iterate over previous paths to add nodes
            for index, path in enumerate(paths):
                if origin == path[-1]:
                    paths[index] = path + [end_l]
                    paths.append(path + [end_r])

            # Initialize path in first iteration
            if i == 0:
                paths.append([i, children_left[i]])
                paths.append([i, children_right[i]])

In [21]:
from src.utils.utils import XPTable

XPTable().df

,Level,Exp.,Exp. Diff,% to 99
0,1,0,NaN,0.0
1,2,83,83.0,0.0
2,3,174,91.0,0.0
3,4,276,102.0,0.0
4,5,388,112.0,0.0
...,...,...,...,...
122,123,140341028,13230768.0,NaN
123,124,154948977,14607949.0,NaN
124,125,171077457,16128480.0,NaN
125,126,188884740,17807283.0,NaN


In [22]:
from src.utils.utils import XPTable

XPTable().exp_to_level(5000)

20

In [32]:
from src.utils.utils import XPTable

def print_node_info(
    label,
    node_info,
    feature,
    threshold,
    y_multi,
    col_names,
    x_multi: pd.DataFrame,
    samp_to_leaf,
    label_i,
    min_samples=100,
):
    # Get unique labels in the target variable
    labels = sorted(y_multi.unique())

    # Unpack node information
    leaf_index, branch_idx, branch, impurity, n_samples, ntotal, value = node_info

    if n_samples < min_samples:
        return

    # Calculate the percentage of samples in this node
    pct_samples = 100.0 * float(n_samples) / float(ntotal)

    # Generate the file names
    file_stem = f"{label}-{label_i}"
    filename = f"{file_stem}.txt"

    # Create a delimiter for separating sections
    delimiter_text = "-" * 50 + "\n"

    # Sort labels based on values for printing purposes
    label_vals = sorted(zip(labels, value), key=lambda tup: tup[1], reverse=True)

    decision_rules = []

    # Find the sample indices that fall in this leaf node
    samp_idxs_in_leaf = samp_to_leaf.index[samp_to_leaf == leaf_index]
    samp_idx_in_leaf = samp_idxs_in_leaf[0]
    samp_in_leaf = x_multi.iloc[samp_idx_in_leaf]
    samp_idxs_df = x_multi.iloc[samp_idxs_in_leaf]
    out_df: pd.DataFrame = samp_idxs_df.copy()

    # Export the DataFrame with samples in this leaf node to a CSV file
    out_df.to_csv(f"{file_stem}-#{n_samples}.csv")

    xp_table = XPTable()

    # Construct decision rules based on the branch information
    for elem in branch:
        op = "<=" if samp_in_leaf[feature[elem]] <= threshold[elem] else ">"
        column_name = col_names[feature[elem]].lower()

        # print(elem, threshold[elem])
        if threshold[elem] >0:
            level = xp_table.exp_to_level(threshold[elem])
        else:
            level = 0
        
        lvl = "" if column_name not in utils.SKILLS else (f"(lvl {level})")
        s = f"{col_names[feature[elem]]} {op} {threshold[elem]} {lvl}"
        decision_rules.append(s)

    # Write information to the output file
    with open(filename, "w") as f:
        f.write(delimiter_text)
        f.write(f"Branch: {branch_idx}, Path: {branch}\n")
        f.write(f"\t{n_samples} in node ({pct_samples}%) (from training + test data)\n")
        f.write(f"\tGin {impurity} at leaf node {branch[-1]}\n")

        # Write values associated with each label (if value is not 0)
        f.write(f"\tValue (from training data):\n")
        for label, value in label_vals:
            if value == 0:
                continue
            f.write("\t\t" + f"{label}: {value}" + "\n")

        # Write decision rules for the branch
        f.write(f"\tDecision Rules:\n")
        for rule in decision_rules:
            f.write(f"\t\t" + str(rule) + "\n")

        # Check conditions and write additional statistics for 'Unknown_bot' label
        if label == "Unknown_bot":
            print(f"For {filename}: label == 'Unknown_bot'")
            print(f"Printing df.describe() in {filename}")
            f.write(f"\t{out_df.describe()}\n")
        f.write(delimiter_text)
    return

In [25]:
TREE_DEPTH = 20

print(f"##############################################################")
print(f"### Training decision tree with depth {TREE_DEPTH}.#####################")
print(f"### Generating a label-id.csv file for each leaf node, where #")
print(f"### label is the tree's classification at that node, and #####")
print(f"### id is unique for each label. id 0 is the leaf containing #")
print(f"### the most samples, 1 the next most, etc... ################")
print(f"### Also generating corresponding label-id.txt files, ########")
print(f"### containing descriptive info about the .csv file ##########")
print(f"##############################################################")

clf = classifier.DTclassifier(f"DTmultiClassifier{TREE_DEPTH}", max_depth=TREE_DEPTH)
score = fit_score(clf, x_multi, y_multi, rpt=True)

##############################################################
### Training decision tree with depth 20.#####################
### Generating a label-id.csv file for each leaf node, where #
### label is the tree's classification at that node, and #####
### id is unique for each label. id 0 is the leaf containing #
### the most samples, 1 the next most, etc... ################
### Also generating corresponding label-id.txt files, ########
### containing descriptive info about the .csv file ##########
##############################################################
accuracy: 0.8522426639587461, roc_auc: 0.9552847875438966
                  precision    recall  f1-score   support

     Agility_bot       0.98      0.94      0.96       666
     Barrows_bot       0.81      0.76      0.78       244
  Blast_mine_bot       0.63      0.52      0.57       230
     Cooking_bot       0.89      0.86      0.87       234
    Crafting_bot       0.98      0.98      0.98      1185
     Fishing_bot       0.9

In [33]:
# above ~10 is too deep to graph, so print helpful output instead.
col_names = x_multi.columns.tolist()

n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
impurity = clf.tree_.impurity
value = clf.tree_.value
all_branches = list(retrieve_branches(n_nodes, children_left, children_right))
samp_to_leaf = pd.Series(clf.apply(x_multi))
leaf_idxs = pd.Series(samp_to_leaf).value_counts()
ntotal = leaf_idxs.sum()


def get_label_node_info(index, branch, leaf_idxs, impurity, ntotal, value, clf_classes):
    leaf_index = branch[-1]
    n_samples = leaf_idxs[leaf_index]
    node_info = (
        leaf_index,
        index,
        branch,
        impurity[leaf_index],
        n_samples,
        ntotal,
        value[leaf_index][0],
    )
    label = clf_classes[np.argmax(value[leaf_index])]
    return label, node_info


def get_label_to_node_info_dict(all_branches):
    label_to_node_info = collections.defaultdict(list)
    for index, branch in enumerate(all_branches):
        label, node_info = get_label_node_info(
            index, branch, leaf_idxs, impurity, ntotal, value, clf.classes_
        )
        label_to_node_info[label].append(node_info)
    return label_to_node_info


label_to_node_info = get_label_to_node_info_dict(all_branches)
labels = list(label_to_node_info.keys())


def generate_label_file(
    label, node_info, feature, threshold, y_multi, col_names, x_multi, samp_to_leaf
):
    print("###################################################")
    print("### Generating label files for: ", label, "########")
    print("###################################################")
    for i, node_info in enumerate(
        sorted(node_info, key=lambda tup: tup[4], reverse=True)
    ):
        print_node_info(
            label,
            node_info,
            feature,
            threshold,
            y_multi,
            col_names,
            x_multi,
            samp_to_leaf,
            i,
        )


def generate_label_files(
    label_to_node_info, feature, threshold, y_multi, col_names, x_multi, samp_to_leaf
):
    for label, node_info in label_to_node_info.items():
        generate_label_file(
            label,
            node_info,
            feature,
            threshold,
            y_multi,
            col_names,
            x_multi,
            samp_to_leaf,
        )


generate_label_files(
    label_to_node_info, feature, threshold, y_multi, col_names, x_multi, samp_to_leaf
)

###################################################
### Generating label files for:  Real_Player ########
###################################################
###################################################
### Generating label files for:  Mining_bot ########
###################################################
###################################################
### Generating label files for:  PVM_Melee_bot ########
###################################################
###################################################
### Generating label files for:  Runecrafting_bot ########
###################################################
###################################################
### Generating label files for:  Unknown_bot ########
###################################################
###################################################
### Generating label files for:  LMS_bot ########
###################################################
##################################################

In [34]:
s = tree.export_text(
    clf,
    feature_names=x_multi.columns.tolist(),
    max_depth=100,
    show_weights=True,
    spacing=1,
)
with open("inspect_dt-tree.txt", "w") as f:
    f.write(s)

In [35]:
# Graph some decision trees with lower depths.
PLOT_TREES = False
if PLOT_TREES:
    print("Generating small decision tree images (large ones dont display well)")
    scores = []
    i_vals = []
    for i in range(2, 10):
        dt_multi_classifier = classifier.DTclassifier(
            f"DTmultiClassifier{i}", max_depth=i
        )
        score = fit_score(dt_multi_classifier, x_multi, y_multi)
        scores.append(score)
        i_vals.append(i_vals)
        out_file = f"DTmultiClassifer_{i}"
        dot_data = tree.export_graphviz(
            dt_multi_classifier,
            rotate=True,
            impurity=False,
            precision=1,
            feature_names=x_multi.columns,
            class_names=sorted(y_multi.unique()),
        )
        graph = graphviz.Source(dot_data)
        graph.render(filename=out_file, format="pdf")

In [36]:
y_multi.value_counts() / len(y_multi)

label
Real_Player         0.469290
Unknown_bot         0.207945
PVM_Melee_bot       0.064573
Magic_bot           0.036298
Smithing_bot        0.035716
Fishing_bot         0.022980
Crafting_bot        0.020767
Zulrah_bot          0.016713
Mining_bot          0.016604
Fletching_bot       0.016064
Hunter_bot          0.014746
Agility_bot         0.011677
Woodcutting_bot     0.011628
Runecrafting_bot    0.007262
LMS_bot             0.006975
Zalcano_bot         0.006421
Herblore_bot        0.006056
Thieving_bot        0.005958
Barrows_bot         0.004282
Soul_Wars_bot       0.004208
Cooking_bot         0.004096
Blast_mine_bot      0.004040
Wintertodt_bot      0.003261
Vorkath_bot         0.002441
Name: count, dtype: float64